# Version 4.0

- Uses johnsnowlabs healthcare NLP software to link values to the BTO

# REQUIREMENTS
- Requires Java 11 to be installed and selected
- Spark must be installed and spark home variable set

In [1]:
%%capture
import os
import json

with open('spark_nlp_for_healthcare_spark_ocr_8085.json') as f:
    license_keys = json.load(f)

locals().update(license_keys)

for k,v in license_keys.items():
    os.environ[k] = v


In [2]:
%pip -q install --upgrade spark-nlp-jsl==$JSL_VERSION --extra-index-url=https://pypi.johnsnowlabs.com/$SECRET
%pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sparknlp

print("Spark NLP version", sparknlp.version())
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp.base import *
import pyspark.sql.functions as F

# import internal package
import sparknlp_jsl
from sparknlp_jsl.annotator import *

print("sparknlp-jsl veriosn: ", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'], aarch64=True)

Spark NLP version 5.1.1
sparknlp-jsl veriosn:  5.1.1


:: loading settings :: url = jar:file:/home/ec2-user/workspace/.johnsnowlabs/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-aarch64_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e82cce6-129d-4f2b-ab43-d7b71e87a50d;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-aarch64_2.12;5.1.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com

In [35]:
from sparknlp.pretrained import PretrainedPipeline

pipeline = PretrainedPipeline('explain_clinical_doc_carp', 'en', 'clinical/models')

pipeline.model.stages

explain_clinical_doc_carp download started this may take some time.
Approx size to download 1.6 GB
[OK!]


[DocumentAssembler_9619f8fd837c,
 SentenceDetector_c0b14c755033,
 REGEX_TOKENIZER_bcb8fead005d,
 POS_6f55785005bf,
 dependency_d5a8da6c9093,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_cd5ce67b529f,
 NER_CONVERTER_2f1dcb61b142,
 MedicalNerModel_4a303d875127,
 NER_CONVERTER_a8cff4d56af8,
 ASSERTION_DL_25881ab6309e,
 RelationExtractionModel_9c255241fec3]

In [42]:
#  store the bert_token_classifier_ner_jsl model in the pipeline
# get the embeddings of the model
print(type(pipeline.model.stages[8]))
model = pipeline.model.stages[6]

#  get all parameters of the model
model.extractParamMap()

#  get the embeddings of the model
# model.explainParam('inputCols')

# MedicalNerModel.getEmbeddingsSource(pipeline.model.stages[8])
# pipeline.model.stages[8].getEmbeddings()

<class 'sparknlp_jsl.annotator.ner.medical_ner.MedicalNerModel'>


{Param(parent='MedicalNerModel_cd5ce67b529f', name='batchSize', doc='Size of every batch'): 64,
 Param(parent='MedicalNerModel_cd5ce67b529f', name='includeAllConfidenceScores', doc='whether to include all confidence scores in annotation metadata or just the score of the predicted tag'): False,
 Param(parent='MedicalNerModel_cd5ce67b529f', name='includeConfidence', doc='whether to include confidence scores in annotation metadata'): True,
 Param(parent='MedicalNerModel_cd5ce67b529f', name='inferenceBatchSize', doc='number of sentences to process in a single batch during inference'): 1,
 Param(parent='MedicalNerModel_cd5ce67b529f', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='MedicalNerModel_cd5ce67b529f', name='sentenceTokenIndex', doc='Whether to include the token index for each sentence in annotation metadata.'): False,
 Param(parent='MedicalNerModel_cd5ce67b529f', name='labelCasing', doc='Setting all labels of the N